In [60]:
!pip install faiss-cpu
!pip install sentence-transformers

In [2]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain.llms import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [3]:
#load api key
#os.environ[''] = 'your_openai_api_key'

In [4]:
!pip install openai

In [5]:
llm = OpenAI(temperature=0.9, max_tokens=500)

loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/personal-finance/",
    "https://www.moneycontrol.com/news/fintech/"
])
data = loaders.load()
len(data)

C:\Users\visha\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


2

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap = 200
)

docs = text_splitter.split_documents(data)
len(docs)

44

In [7]:
docs[9]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/personal-finance/'}, page_content='ForumEXPLORE FORUMHomepageMembership RulesForum TopicsAsk the ExpertTop BoardersUSER PROFILEFORUM TOPICSLatest ThreadsStocksIndexGoldPersonal FinanceJust PostedMF FORUMPOLLSLatest PollsHistorical PollsMarket SentimentsSUPPORTFAQsCode of ConductFeedbackWrite to us\n\nVideosVIDEOSHomepageVideos on DemandMarkets with Santo & CJMorning TradeCommodities chat with Manisha GuptaLet`s Talk JobsThe TenantDrive ReportBajar GupshupBits To BillionsPODCASTHomepagePodcast on DemandThe Week on Dalal StreetMarket MinutesMC Special PodcastSimply SavePolicy TalksLIVE TVHindiGujarati')

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Define the file path
file_path = "faiss_store"

# Assuming `docs` is already defined
embeddings = OpenAIEmbeddings()
vectorindex_openai = FAISS.from_documents(docs, embeddings)

# Save the FAISS index to disk
vectorindex_openai.save_local(file_path)

In [13]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI

# Define the file path
file_path = "faiss_store"

# Load the FAISS index from disk
vectorindex_openai = FAISS.load_local(file_path, OpenAIEmbeddings(), allow_dangerous_deserialization=True)

# Set up your LLM (assuming you're using OpenAI LLM)
llm = OpenAI(api_key="your_openai_api_key")

# Create the RetrievalQAWithSourcesChain
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex_openai.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x0000018BE82B2790>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000018BE8549460>, openai_api_key='sk-proj-WVUv1C9wUplTFeMp5SDvT3BlbkFJtZ3W8q6qmDIB381j6niQ', openai_proxy='')), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that y

In [75]:
query = "news from SEBI"

langchain.debug=True

chain({"question": query}, return_only_outputs = True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "news from SEBI"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Invest NowEXPERT TRADING GUIDANCEPowered By Trading Advisories, Courses & Webinars by Top Verified Experts.Unlock Your Trading Potential: Trade like Experts with SEBI registered creators, Learn from Courses & Webinars by India's Finest Finance Experts.Invest NowINVEST IN CRYPTOCURRENCIESPowered By Invest in top cryptocurrencies all over the worldSee the top gainers, losers, invest and get updated what's happening in the crypto marketInvest Now\n\nPRO\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHome\n\nNews\n\nFintech\n\nGenA

[llm/end] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OpenAI] [1.46s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Unlock Your Trading Potential: Trade like Experts with SEBI registered creators, Learn from Courses & Webinars by India's Finest Finance Experts.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 93,
      "prompt_tokens": 864,
      "total_tokens": 957
    },
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[llm/end] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OpenAI] [1.46s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Unlock Your Trading Potential: Trade like Experts with SEBI registered creators, Learn from Courses & Webinars 

[llm/end] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:OpenAI] [1.35s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " There is no relevant news from SEBI reported in these sources. \nSOURCES: https://www.moneycontrol.com/news/fintech/, https://www.moneycontrol.com/news/business/personal-finance/",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 41,
      "prompt_tokens": 1505,
      "total_tokens": 1546
    },
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}
[chain/end] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] [1.35s] Exiting Chain run with output:
{
  "text": " There is no relevant news from SEBI reported in these sources. \nSOURCES: https://www.moneycontrol.com/news/fintech/, https:

{'answer': ' There is no relevant news from SEBI reported in these sources. \n',
 'sources': 'https://www.moneycontrol.com/news/fintech/, https://www.moneycontrol.com/news/business/personal-finance/'}